## Data Preprocessing

### Project Overview

- This notebook is part of a project aimed at building an AI-powered semantic search tool for Nike products. 
- The goal is to leverage natural language processing techniques to enable intuitive and accurate product recommendations from a dataset of Nike products.

### Why This Dataset?
I selected this dataset because:
- It contains detailed product information, including titles, descriptions, reviews, prices, and availability, which are essential for building a semantic search engine.
- It includes structured and semi-structured data that allow for feature extraction and preprocessing to support natural language processing (NLP) tasks.
- The dataset provides real-world e-commerce data, making it applicable for developing a practical recommendation system.
